In [ ]:
from models.swarm_job import SwarmJob, RunStrategy
import generate_all_maximum_projections

In [ ]:
# Set strategy here -- "RunStrategy.SWARM" is running on Biowulf/with swarm in parallel, "RunStrategy.LOCAL" is testing a single well/image without parallelization

SwarmJob.run_strategy = RunStrategy.LOCAL

In [ ]:
# Set your file system variables here

source_images = "C:\\Users\\finne\\tktktk"
results_directory = Path("C:\\Users\\finne\\tktktk\\something_else")
MIP_directory = # add MIP to results directory


In [ ]:
# Maximum projections!
GenerateAllMaximumProjectionsJob(source_images, maxproj_zcenters_dir).run()